Train an Encoder–Decoder model that can convert a date string from one format to another (e.g., from "April 22, 2019" to "2019-04-22").

## Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Let's start by creating the dataset. We will use random days between 1000-01-01 and 9999-12-31.

In [10]:
from datetime import date

MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]


def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()
    
    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]
    
    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

In [11]:
date(1000, 1, 1).toordinal()

364878

In [12]:
date(9999, 12, 31).toordinal()

3652059

In [13]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-"*50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


Let's get the list of all possible characters in the inputs:

In [14]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS)))) + "0123456789, "
INPUT_CHARS

'ADFJMNOSabceghilmnoprstuvy0123456789, '

And here's the list of possible characters in the outputs.

In [15]:
OUTPUT_CHARS = "0123456789-"

In [16]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [17]:
date_str_to_ids(x_example[0], INPUT_CHARS)

[7, 11, 19, 22, 11, 16, 9, 11, 20, 37, 28, 26, 36, 37, 33, 26, 33, 31]

In [18]:
date_str_to_ids(y_example[0], OUTPUT_CHARS)

[7, 0, 7, 5, 10, 0, 9, 10, 2, 0]

In [19]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor()  # using 0 as the padding token ID

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [20]:
np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [21]:
Y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1])>

### First Version: a very basic seq2seq model

Let's first try the simplest possible model: we feed in the input sequence, which first goes through the encoder (an embedding layer followed by a single LSTM layer), which outputs a vector, then it goes through a decoder (a single LSTM layer, followed by a dense output layer), which outputs a sequence of vectors, each representing the estimated probabilities for all possible output character.

Since the decoder expects a sequence as input, we repeat the vector (which is output by the decoder) as many times as the longest possible output sequence.

In [26]:
embedding_size = 32
max_output_length = Y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim = len(INPUT_CHARS) +1,
                           output_dim = embedding_size,
                           input_shape = [None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")
])

model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(max_output_length),
    decoder
])

optimizer = keras.optimizers.Nadam()
model.compile(loss = "sparse_categorical_crossentropy", optimizer=optimizer,
             metrics = ["accuracy"])
history = model.fit(X_train, Y_train, epochs = 20,
                   validation_data = (X_valid, Y_valid))

Epoch 1/20
313/313 [==============================] - 4s 12ms/step - loss: 1.8238 - accuracy: 0.3444 - val_loss: 1.3930 - val_accuracy: 0.4723
Epoch 2/20
313/313 [==============================] - 3s 9ms/step - loss: 1.3433 - accuracy: 0.5123 - val_loss: 1.2283 - val_accuracy: 0.5559
Epoch 3/20
313/313 [==============================] - 3s 8ms/step - loss: 1.0633 - accuracy: 0.6173 - val_loss: 0.9442 - val_accuracy: 0.6590
Epoch 4/20
313/313 [==============================] - 3s 9ms/step - loss: 0.8061 - accuracy: 0.7016 - val_loss: 0.8413 - val_accuracy: 0.6798
Epoch 5/20
313/313 [==============================] - 3s 9ms/step - loss: 0.6912 - accuracy: 0.7490 - val_loss: 0.5185 - val_accuracy: 0.7995
Epoch 6/20
313/313 [==============================] - 3s 9ms/step - loss: 0.5344 - accuracy: 0.8026 - val_loss: 0.4014 - val_accuracy: 0.8424
Epoch 7/20
313/313 [==============================] - 3s 9ms/step - loss: 0.3334 - accuracy: 0.8748 - val_loss: 0.2732 - val_accuracy: 0.9011
Epoch

In [27]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence])
           for sequence in ids]

In [28]:
X_new = prepare_date_strs(["September 17, 2009", "July 14, 1789"])

In [29]:
ids = model.predict_classes(X_new)
for date_str in ids_to_date_strs(ids):
    print(date_str)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
2009-09-17
1789-07-14


Perfect! :)

However, since the model was only trained on input strings of length 18 (which is the length of the longest date), it does not perform well if we try to use it to make predictions on shorter sequences:

In [30]:
X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"])

In [31]:
ids = model.predict_classes(X_new)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2020-01-02
1789-09-14


We need to ensure that we always pass sequences of the same length as during training, using padding if necessary

In [32]:
max_input_length = X_train.shape[1]

def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        #[[top, bottom],[left, right]]
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = model.predict_classes(X)
    return ids_to_date_strs(ids)

In [34]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

['2020-05-02', '1789-07-14']

### Second version: feeding the shifted targets to the decoder (teacher forcing)

Instead of feeding the decoder a simple repetition of the encoder's output vector, we can feed it the target sequence, shifted by one time step to the right. This way, at each time step the decoder will know what the previous target character was. This should help is tackle more complex sequence-to-sequence problems.

Since the first output character of each target sequence has no previous character, we will need a new token to represent the start-of-sequence (sos).

During inference, we won't know the target, so what will we feed the decoder? We can just predict one character at a time, starting with an sos token, then feeding the decoder all the characters that were predicted so far (we will look at this in more details later in this notebook).

But if the decoder's LSTM expects to get the previous target as input at each step, how shall we pass the vector output by the encoder? Well, one option is to ignore the output vector, and instead use the encoder's LSTM state as the initial state of the decoder's LSTM (which requires that encoder's LSTM must have the same number of units as the decoder's LSTM).

Now let's create the decoder's inputs (for training, validation and testing). The sos token will be represented using the last possible output character's ID + 1.

In [36]:
sos_id = len(OUTPUT_CHARS) + 1

def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims =(len(Y), 1), value = sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis = 1)

X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

In [37]:
X_train_decoder

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[12,  8,  1, ..., 10, 11,  3],
       [12,  9,  6, ...,  6, 11,  2],
       [12,  8,  2, ...,  2, 11,  2],
       ...,
       [12, 10,  8, ...,  2, 11,  4],
       [12,  2,  2, ...,  3, 11,  3],
       [12,  8,  9, ...,  8, 11,  3]])>

In [38]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = keras.layers.Embedding(
    input_dim =len(INPUT_CHARS) + 1,
    output_dim = encoder_embedding_size)(encoder_input)
_, encoder_state_h, encoder_state_c = keras.layers.LSTM(
    lstm_units, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = keras.layers.Embedding(
    input_dim=len(OUTPUT_CHARS) + 2,
    output_dim=decoder_embedding_size)(decoder_input)
decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences = True)(
    decoder_embedding, initial_state=encoder_state)
decoder_output = keras.layers.Dense(len(OUTPUT_CHARS) +1,
                                   activation = "softmax")(decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input],
                          outputs=[decoder_output])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
             metrics=["accuracy"])
history = model.fit([X_train, X_train_decoder], Y_train, epochs = 10,
                   validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/10
313/313 [==============================] - 4s 12ms/step - loss: 1.6848 - accuracy: 0.3746 - val_loss: 1.4282 - val_accuracy: 0.4592
Epoch 2/10
313/313 [==============================] - 3s 8ms/step - loss: 1.1913 - accuracy: 0.5624 - val_loss: 0.9003 - val_accuracy: 0.6798
Epoch 3/10
313/313 [==============================] - 3s 8ms/step - loss: 0.5973 - accuracy: 0.7932 - val_loss: 0.3314 - val_accuracy: 0.9041
Epoch 4/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2083 - accuracy: 0.9529 - val_loss: 0.0916 - val_accuracy: 0.9906
Epoch 5/10
313/313 [==============================] - 3s 8ms/step - loss: 0.0688 - accuracy: 0.9930 - val_loss: 0.0383 - val_accuracy: 0.9987
Epoch 6/10
313/313 [==============================] - 3s 8ms/step - loss: 0.0535 - accuracy: 0.9934 - val_loss: 0.0240 - val_accuracy: 0.9997
Epoch 7/10
313/313 [==============================] - 3s 8ms/step - loss: 0.0162 - accuracy: 0.9999 - val_loss: 0.0132 - val_accuracy: 0.9998
Epoch

In [45]:
sos_id = len(OUTPUT_CHARS) + 1

def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=(len(X), 1), value = sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - Y_pred.shape[1]
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, pad_size]])
        Y_probas_next = model.predict([X, X_decoder])[:, index:index+1]
        Y_pred_next = tf.argmax(Y_probas_next, axis = -1, output_type = tf.int32)
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis = 1)
    return ids_to_date_strs(Y_pred[:, 1:])

In [46]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

### Third version: using TF-Addons's seq2seq implementation

Let's build exactly the same model, but using TF-Addon's seq2seq API. The implementation below is almost very similar to the TFA example higher in this notebook, except without the model input to specify the output sequence length, for simplicity (but you can easily add it back in if you need it for your projects, when the output sequences have very different lengths).

In [48]:
import tensorflow_addons as tfa

np.random.seed(42)
tf.random.set_seed(42)

encoder_embedding_size = 32
decoder_embedding_size = 32
units = 128

encoder_inputs = keras.layers.Input(shape = [None], dtype = np.int32)
decoder_inputs = keras.layers.Input(shape = [None], dtype = np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

encoder_embeddings = keras.layers.Embedding(
    len(INPUT_CHARS) + 1, encoder_embedding_size)(encoder_inputs)

decoder_embedding_layer = keras.layers.Embedding(
    len(INPUT_CHARS) + 2, decoder_embedding_size)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(units)
output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell,
                                                sampler,
                                                output_layer=output_layer)

final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings,
    initial_state = encoder_state)

Y_proba = keras.layers.Activation("softmax")(final_outputs.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs,  decoder_inputs],
                              outputs=[Y_proba])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
             metrics = ["accuracy"])
history = model.fit([X_train, X_train_decoder], Y_train, epochs=15,
                   validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/15
313/313 [==============================] - 11s 35ms/step - loss: 1.6793 - accuracy: 0.3668 - val_loss: 1.4547 - val_accuracy: 0.4364
Epoch 2/15
313/313 [==============================] - 10s 32ms/step - loss: 1.3875 - accuracy: 0.4571 - val_loss: 1.3003 - val_accuracy: 0.4942
Epoch 3/15
313/313 [==============================] - 10s 33ms/step - loss: 1.0189 - accuracy: 0.6279 - val_loss: 0.7626 - val_accuracy: 0.7342
Epoch 4/15
313/313 [==============================] - 10s 33ms/step - loss: 0.4980 - accuracy: 0.8302 - val_loss: 0.2509 - val_accuracy: 0.9377
Epoch 5/15
313/313 [==============================] - 10s 33ms/step - loss: 0.1789 - accuracy: 0.9641 - val_loss: 0.0840 - val_accuracy: 0.9920
Epoch 6/15
313/313 [==============================] - 10s 32ms/step - loss: 0.0549 - accuracy: 0.9967 - val_loss: 0.0378 - val_accuracy: 0.9984
Epoch 7/15
313/313 [==============================] - 10s 32ms/step - loss: 0.0491 - accuracy: 0.9939 - val_loss: 0.0259 - val_accuracy:

In [49]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

However, there's a much more efficient way to perform inference. Until now, during inference, we've run the model once for each new character. Instead, we can create a new decoder, based on the previously trained layers, but using a `GreedyEmbeddingSampler` instead of a `TrainingSampler`.

At each time step, the `GreedyEmbeddingSampler` will compute the argmax of the decoder's outputs, and run the resulting token IDs through the decoder's embedding layer. Then it will feed the resulting embeddings to the decoder's LSTM cell at the next time step. This way, we only need to run the decoder once to get the full prediction.

In [50]:
inference_sampler = tfa.seq2seq.sampler.GreedyEmbeddingSampler(
    embedding_fn = decoder_embedding_layer)

inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
    decoder_cell, inference_sampler, output_layer=output_layer,
    maximum_iterations=max_output_length)

batch_size = tf.shape(encoder_inputs)[:1]
start_tokens = tf.fill(dims=batch_size, value=sos_id)
final_outputs, final_state, final_sequence_lengths = inference_decoder(
    start_tokens,
    initial_state=encoder_state,
    start_tokens=start_tokens,
    end_token=0)

inference_model = keras.models.Model(inputs=[encoder_inputs],
                                    outputs=[final_outputs.sample_id])

A few notes:
* The `GreedyEmbeddingSampler` needs the `start_tokens` (a vector containing the start-of-sequence ID for each decoder sequence), and the `end_token` (the decoder will stop decoding a sequence once the model outputs this token).
* We must set `maximum_iterations` when creating the `BasicDecoder`, or else it may run into an infinite loop (if the model never outputs the end token for at least one of the sequences). This would force you would to restart the Jupyter kernel.
* The decoder inputs are not needed anymore, since all the decoder inputs are generated dynamically based on the outputs from the previous time step.
* The model's outputs are `final_outputs.sample_id` instead of the softmax of `final_outputs.rnn_outputs`. This allows us to directly get the argmax of the model's outputs. If you prefer to have access to the logits, you can replace `final_outputs.sample_id` with `final_outputs.rnn_outputs`.

In [51]:
def fast_predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = inference_model.predict(X)
    return ids_to_date_strs(Y_pred)

In [52]:
fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

In [53]:
%timeit predict_date_strs(["July 14, 1789", "May 01, 2020"])

597 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%timeit fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])

59 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Fourth version: using TF-Addons's seq2seq implementation with a scheduled sampler

When we trained the previous model, at each time step _t_ we gave the model the target token for time step _t_ - 1. However, at inference time, the model did not get the previous target at each time step. Instead, it got the previous prediction. So there is a discrepancy between training and inference, which may lead to disappointing performance. To alleviate this, we can gradually replace the targets with the predictions, during training. For this, we just need to replace the `TrainingSampler` with a `ScheduledEmbeddingTrainingSampler`, and use a Keras callback to gradually increase the `sampling_probability` (i.e., the probability that the decoder will use the prediction from the previous time step rather than the target for the previous time step).

In [57]:
import tensorflow_addons as tfa

np.random.seed(42)
tf.random.set_seed(42)

n_epochs = 20
encoder_embedding_size = 32
decoder_embedding_size = 32
units = 128

encoder_inputs = keras.layers.Input(shape = [None], dtype = np.int32)
decoder_inputs = keras.layers.Input(shape = [None], dtype = np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

encoder_embeddings = keras.layers.Embedding(
    len(INPUT_CHARS) + 1, encoder_embedding_size)(encoder_inputs)

decoder_embedding_layer = keras.layers.Embedding(
    len(INPUT_CHARS) + 2, decoder_embedding_size)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.ScheduledEmbeddingTrainingSampler(
    sampling_probability=0.,
    embedding_fn=decoder_embedding_layer)

sampler.sampling_probability = tf.Variable(0.)

decoder_cell = keras.layers.LSTMCell(units)
output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell,
                                                sampler,
                                                output_layer=output_layer)

final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings,
    initial_state = encoder_state)

Y_proba = keras.layers.Activation("softmax")(final_outputs.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs,  decoder_inputs],
                              outputs=[Y_proba])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
             metrics = ["accuracy"])

def update_sampling_probability(epoch, logs):
    proba = min(1.0, epoch / (n_epochs - 10))
    sampler.sampling_probability.assign(proba)

sampling_probability_cb = keras.callbacks.LambdaCallback(
    on_epoch_begin=update_sampling_probability)
    
history = model.fit([X_train, X_train_decoder], Y_train, epochs=n_epochs,
                   validation_data=([X_valid, X_valid_decoder], Y_valid),
                   callbacks=[sampling_probability_cb])

Epoch 1/20


C:\Users\G Surya Krishna\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


313/313 [==============================] - 17s 53ms/step - loss: 1.6793 - accuracy: 0.3668 - val_loss: 1.4547 - val_accuracy: 0.4366
Epoch 2/20
313/313 [==============================] - 17s 54ms/step - loss: 1.3882 - accuracy: 0.4603 - val_loss: 1.2549 - val_accuracy: 0.5329
Epoch 3/20
313/313 [==============================] - 17s 53ms/step - loss: 1.0127 - accuracy: 0.6265 - val_loss: 0.7641 - val_accuracy: 0.7214
Epoch 4/20
313/313 [==============================] - 16s 51ms/step - loss: 0.5738 - accuracy: 0.7882 - val_loss: 0.3952 - val_accuracy: 0.8627
Epoch 5/20
313/313 [==============================] - 15s 49ms/step - loss: 0.3526 - accuracy: 0.8887 - val_loss: 0.2433 - val_accuracy: 0.9267
Epoch 6/20
313/313 [==============================] - 16s 51ms/step - loss: 0.1939 - accuracy: 0.9459 - val_loss: 0.1391 - val_accuracy: 0.9628
Epoch 7/20
313/313 [==============================] - 15s 49ms/step - loss: 0.1142 - accuracy: 0.9720 - val_loss: 0.0945 - val_accuracy: 0.9775
Epo

For inference, we could do the exact same thing as earlier, using a `GreedyEmbeddingSampler`. However, just for the sake of completeness, let's use a `SampleEmbeddingSampler` instead. It's almost the same thing, except that instead of using the argmax of the model's output to find the token ID, it treats the outputs as logits and uses them to sample a token ID randomly. This can be useful when you want to generate text. The `softmax_temperature` argument serves the 
same purpose as when we generated Shakespeare-like text (the higher this argument, the more random the generated text will be).

In [58]:
softmax_temperature = tf.Variable(1.)

inference_sampler = tfa.seq2seq.sampler.SampleEmbeddingSampler(
    embedding_fn = decoder_embedding_layer,
    softmax_temperature=softmax_temperature)

inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
    decoder_cell, inference_sampler, output_layer=output_layer,
    maximum_iterations=max_output_length)

batch_size = tf.shape(encoder_inputs)[:1]
start_tokens = tf.fill(dims=batch_size, value=sos_id)
final_outputs, final_state, final_sequence_lengths = inference_decoder(
    start_tokens,
    initial_state = encoder_state,
    start_tokens = start_tokens,
    end_token = 0
)

inference_model = keras.models.Model(inputs=[encoder_inputs],
                                    outputs=[final_outputs.sample_id])

In [59]:
def creative_predict_date_strs(date_strs, temperature=1.0):
    softmax_temperature.assign(temperature)
    X = prepare_date_strs_padded(date_strs)
    Y_pred = inference_model.predict(X)
    return ids_to_date_strs(Y_pred)

In [60]:
tf.random.set_seed(42)

creative_predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

In [63]:
tf.random.set_seed(42)

creative_predict_date_strs(["July 14, 1789", "May 01, 2020"],
                           temperature=5.)

['1479307-19', '200040?400']

### Fifth version: using TFA seq2seq, the Keras subclassing API and attention mechanisms

The sequences in this problem are pretty short, but if we wanted to tackle longer sequences, we would probably have to use attention mechanisms. While it's possible to code our own implementation, it's simpler and more efficient to use TF-Addons's implementation instead. Let's do that now, this time using Keras' subclassing API.


In this implementation, we've reverted back to using the `TrainingSampler`, for simplicity (but you can easily tweak it to use a `ScheduledEmbeddingTrainingSampler` instead). We also use a `GreedyEmbeddingSampler` during inference, so this class is pretty easy to use:

In [75]:
class DateTranslation(keras.models.Model):
    def __init__(self, units=128, encoder_embedding_size=32,
                decoder_embedding_size=32, **kwargs):
        super().__init__(**kwargs)
        self.encoder_embedding = keras.layers.Embedding(
            input_dim = len(INPUT_CHARS) + 1,
            output_dim = encoder_embedding_size)
        self.encoder = keras.layers.LSTM(units,
                                        return_sequences=True,
                                        return_state=True)
        self.decoder_embedding = keras.layers.Embedding(
            input_dim = len(OUTPUT_CHARS) + 2,
            output_dim = decoder_embedding_size)
        self.attention = tfa.seq2seq.LuongAttention(units)
        decoder_inner_cell = keras.layers.LSTMCell(units)
        self.decoder_cell = tfa.seq2seq.AttentionWrapper(
            cell=decoder_inner_cell,
            attention_mechanism=self.attention)
        output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)
        self.decoder = tfa.seq2seq.BasicDecoder(
            cell=self.decoder_cell,
            sampler=tfa.seq2seq.sampler.TrainingSampler(),
            output_layer=output_layer)
        self.inference_decoder = tfa.seq2seq.BasicDecoder(
            cell=self.decoder_cell,
            sampler=tfa.seq2seq.sampler.GreedyEmbeddingSampler(
                embedding_fn = self.decoder_embedding),
            output_layer=output_layer,
            maximum_iterations=max_output_length)
        
    def call(self, inputs, training=None):
        encoder_input, decoder_input = inputs
        encoder_embeddings = self.encoder_embedding(encoder_input)
        encoder_outputs, encoder_state_h, encoder_state_c = self.encoder(
            encoder_embeddings,
            training=training)
        encoder_state = [encoder_state_h, encoder_state_c]
        
        self.attention(encoder_outputs, setup_memory=True)
        
        decoder_embeddings = self.decoder_embedding(decoder_input)
        
        decoder_initial_state = self.decoder_cell.get_initial_state(
            decoder_embeddings)
        decoder_initial_state = decoder_initial_state.clone(
            cell_state=encoder_state)
        
        if training:
            decoder_outputs, _, _  = self.decoder(
                decoder_embeddings,
                initial_state = decoder_initial_state,
                training=training)
        else:
            start_tokens = tf.zeros_like(encoder_input[:,0]) + sos_id
            decoder_outputs, _, _ = self.inference_decoder(
                decoder_embeddings,
                initial_state=decoder_initial_state,
                start_tokens=start_tokens,
                end_token=0)
        return tf.nn.softmax(decoder_outputs.rnn_output)

In [76]:
np.random.seed(42)
tf.random.set_seed(42)

model = DateTranslation()
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
             metrics=["accuracy"])
history = model.fit([X_train, X_train_decoder], Y_train, epochs = 25,
                   validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/25
313/313 [==============================] - 16s 52ms/step - loss: 2.1425 - accuracy: 0.2302 - val_loss: 2.1138 - val_accuracy: 0.2379
Epoch 2/25
313/313 [==============================] - 15s 48ms/step - loss: 1.8695 - accuracy: 0.3229 - val_loss: 2.1359 - val_accuracy: 0.2230
Epoch 3/25
313/313 [==============================] - 15s 47ms/step - loss: 1.5347 - accuracy: 0.4513 - val_loss: 1.3687 - val_accuracy: 0.4842
Epoch 4/25
313/313 [==============================] - 15s 47ms/step - loss: 1.3201 - accuracy: 0.5095 - val_loss: 1.2511 - val_accuracy: 0.5235
Epoch 5/25
313/313 [==============================] - 15s 47ms/step - loss: 1.1755 - accuracy: 0.5569 - val_loss: 1.1243 - val_accuracy: 0.5739
Epoch 6/25
313/313 [==============================] - 15s 48ms/step - loss: 1.0999 - accuracy: 0.5838 - val_loss: 1.1195 - val_accuracy: 0.5836
Epoch 7/25
313/313 [==============================] - 15s 47ms/step - loss: 1.0446 - accuracy: 0.6056 - val_loss: 1.0075 - val_accuracy:

In [79]:
def fast_predict_date_strs_v2(date_strs):
    X = prepare_date_strs_padded(date_strs)
    X_decoder = tf.zeros(shape=(len(X), max_output_length), dtype=tf.int32)
    Y_probas = model.predict([X, X_decoder])
    Y_predict = tf.argmax(Y_probas, axis = -1)
    return ids_to_date_strs(Y_predict)

In [80]:
fast_predict_date_strs_v2(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']